In [1]:
import pandas as pd
import numpy as np

In [334]:
time_series = pd.read_csv('/home/ananth/Downloads/time_series_with_causes_zscore_full.csv')

In [335]:
time_series.columns.values

array(['Unnamed: 0', 'index', 'country', 'admin_code', 'admin_name',
       'centx', 'centy', 'year_month', 'year', 'month', 'fews_ipc',
       'fews_ha', 'fews_proj_near', 'fews_proj_near_ha', 'fews_proj_med',
       'fews_proj_med_ha', 'ndvi_mean', 'ndvi_anom', 'rain_mean',
       'rain_anom', 'et_mean', 'et_anom', 'acled_count',
       'acled_fatalities', 'p_staple_food', 'area', 'cropland_pct', 'pop',
       'ruggedness_mean', 'pasture_pct', 'change_fews', 'land seizures_0',
       'land seizures_1', 'land seizures_2', 'slashed export_0',
       'slashed export_1', 'slashed export_2', 'price rise_0',
       'price rise_1', 'price rise_2', 'mass hunger_0', 'mass hunger_1',
       'mass hunger_2', 'cyclone_0', 'cyclone_1', 'cyclone_2',
       'failed crops_0', 'failed crops_1', 'failed crops_2',
       'disruption to farming_0', 'disruption to farming_1',
       'disruption to farming_2', 'massive starvation_0',
       'massive starvation_1', 'massive starvation_2',
       'abnormall

In [336]:
t_variant_traditional_factors = ['ndvi_mean', 'ndvi_anom', 'rain_mean', 'rain_anom', 'et_mean', 'et_anom', 
                                    'acled_count', 'acled_fatalities', 'p_staple_food']
t_invariant_traditional_factors = ['area', 'cropland_pct', 'pop', 'ruggedness_mean', 'pasture_pct']
news_factors = [name for name in time_series.columns.values if '_0' in name]

In [337]:
len(news_factors)

167

In [6]:
def get_lagged(x, f, t):
    admin_code = x['admin_code']
    year = x['year']
    month = x['month']
    l_month = ((month-1-t)%12)+1
    l_year = year
    if month-t<=0:
        l_year -= 1
    ts=time_series[time_series['admin_code']==admin_code]
    lagged_year_month = '{}_{}'.format(l_year, l_month)
    if lagged_year_month in ts['year_month'].values:
        ts = ts[ts['year_month']==lagged_year_month]
        return ts[f].values[0]
    else:
        return x[f]
    

In [7]:
def add_time_lagged(features, start=3, end=9, diff=1):    
    for f in features:
        for t in range(start,end,diff):
            if '{}_{}'.format(f,t) in time_series:
                continue
            time_series['{}_{}'.format(f,t)] = time_series.apply(lambda x: get_lagged(x, f, t), axis=1)

In [12]:
admins = pd.read_csv('./famine-country-province-district-years-CS.csv')

In [15]:
len(admins.country.unique())

39

In [19]:
admin_names = time_series['admin_name'].unique()
districts = admins['district'].unique()
provinces = admins['province'].unique()
countries = admins['country'].unique()

In [28]:
print (len(admin_names), len(districts), len(provinces), len(countries))
print (len(set(admin_names).difference(districts)))
missing_admin_names = set(admin_names).difference(districts)
print (len(missing_admin_names.difference(provinces)))
missing_admin_names = missing_admin_names.difference(provinces)

(1142, 4113, 474, 39)
369
230


{'Abu Hamad',
 'Abu Jubaiyah',
 'Acul Du Nord',
 "Ad Dali'",
 'Ad Damazin',
 'Ad Damer',
 'Ad Dinder',
 'Ad Douiem',
 'Adan',
 'Adan Yabaal',
 'Addabah',
 'Addis Adaba',
 'Agnuak',
 'Agui\xc3\xa9',
 'Al Deain',
 'Al Fasher',
 'Al Faw',
 'Al Fushqa',
 'Al Gadaref',
 'Al Galabat',
 'Al Gash',
 'Al Geneina',
 'Al Gutaina',
 'Al Jabalian',
 'Al Kamlin',
 'Al Kurumik',
 "Al Ma'afir",
 'Al Mahagil',
 "Al Marawi'ah",
 'Al Rahd',
 'Al Roseires',
 "Al Wazi'iyah",
 "Amanat Al 'Asimah",
 'Amran',
 'Anse-A-Veau',
 "Anse-D'Ainault",
 'As Salam',
 "At Ta'izziyah",
 'Awi/Agew',
 'Bale.1',
 'Balleyara',
 'Banadir',
 'Bandarbeyla',
 'Bankilar\xc3\xa9',
 'Barh El Gazel Nord',
 'Barh El Gazel Ouest',
 'Barh El Gazel Sud',
 'Barh-K\xc3\xb4h',
 'Baw',
 'Baydhaba',
 'Beitbridge',
 'Belbedji',
 'Belet Weyne',
 'Belet Xaawo',
 'Beni',
 'Berber',
 'Bindura',
 'Bossaso',
 'Bukavu',
 'Bulilima (North)',
 'Bulo Burto',
 "Bura'",
 'Buret',
 'Burtinle',
 'Busia.1',
 'Butembo',
 'Butere Mumias',
 'Cayes',
 'Caynabo'

In [200]:
import editdistance
from fuzzywuzzy import fuzz
def find_matching(missing, names):
    matching_districts = {}
    for m in missing:
        max_overlap = 0
        nearest_d = None
        for d in names:
            d = str(d)
            dist = fuzz.partial_ratio(m, d)
            if dist > max_overlap:
                max_overlap = dist
                nearest_d = d
        matching_districts[m] = nearest_d
    return matching_districts


matching = find_matching(missing_admin_names, districts)
matching_p = find_matching(missing_admin_names, provinces)
#manually verify matching and update
for k in matching.keys():
    print (k, matching[k], matching_p[k])


KeyboardInterrupt: 

In [338]:
# after validating the matches, the names are logged in this csv file
valid_matching = pd.read_csv('matching_districts.csv')

In [339]:
matched = valid_matching['missing'].unique()
# matched = [bytes(m).decode("unicode_escape") for  m in matched]
missing_admin_names =  [m.decode("unicode_escape").encode('ascii', 'backslashreplace') for m in missing_admin_names]
print (len(missing_admin_names), len(matched))
set(missing_admin_names).difference(matched)

(230, 230)


set()

In [340]:
def find_province(x):
    try:
        if x in districts:
            return admins[admins['district']==x]['province'].values[0]
        elif x in provinces:
            return x
        elif x.decode("unicode_escape").encode('ascii', 'backslashreplace') in matched:
            x = x.decode("unicode_escape").encode('ascii', 'backslashreplace')
            v = valid_matching[valid_matching['missing']==x]
            if v['match'].values[0]=='district':
                x = v['district'].values[0]
                return admins[admins['district']==x]['province'].values[0]
            elif v['match'].values[0]=='province':
                return v['province'].values[0]
    except:
        raise Exception("Province not found for: {}".format(x))

In [341]:
admin_to_province = {}
for a in admin_names:
    try:
        admin_to_province[a] = find_province(a)
    except:
        print (a)

In [342]:
time_series['province'] = time_series['admin_name'].apply(lambda x: admin_to_province[x])

In [300]:
# [d for d in districts if "dali" in str(d).lower()]
# [p for p in provinces if 'dali' in str(p).lower()]
# admins['country'].unique()
# time_series[time_series['admin_name']=='As Salam']
# admins[admins['country']=='Sudan']['district'].unique()

['El Dali', 'Ad Dali`']

In [343]:
def add_agg_factors(features, level='province'):
    grouped_df = time_series.groupby(['year_month', level]).mean()
    for f in features:
        time_series['{}_{}'.format(f, level)] = time_series.apply(lambda x: grouped_df.ix[x['year_month'], x[level]][f], axis=1)

In [ ]:
add_agg_factors(news_factors)

/home/ananth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [ ]:
add_agg_factors(news_factors, level='country')
add_agg_factors(t_variant_traditional_factors, level='province')
add_agg_factors(t_variant_traditional_factors, level='country')
add_agg_factors(t_invariant_traditional_factors, level='province')
add_agg_factors(t_invariant_traditional_factors, level='country')

In [ ]:
time_series.to_csv('agg_province_features.csv')

In [331]:
news_factors

['land seizures',
 'slashed export',
 'price rise',
 'mass hunger',
 'cyclone',
 'failed crops',
 'disruption to farming',
 'massive starvation',
 'abnormally low rainfall',
 'withheld relief',
 'international alarm',
 'reduced national output',
 'oppressive regimes',
 'pests',
 'continued deterioration',
 'forests destroyed',
 'man-made disaster',
 'food insecurity',
 'harvests are devastated',
 'humanitarian situation',
 'economic impoverishment',
 'clan battle',
 'population crisis',
 'aid appeal',
 'weather extremes',
 'anti-western policies',
 'rinderpest',
 'inadequate rainfall',
 'lack of authority',
 'acute hunger',
 'foreign troops',
 'increased external debt',
 'drought',
 'conflict',
 'failed rains',
 'makeshift camps',
 'civilians uprooted',
 'dysfunction',
 'foreign aid',
 'violent suppression',
 'military dictatorship',
 'climatic hazards',
 'migration',
 'land grab',
 'terrorism',
 'bombing campaign',
 'collapsing economy',
 'military junta',
 'climate change',
 'rising 

In [ ]:
add_time_lagged(t_variant_traditional_factors)

In [ ]:
add_time_lagged(news_factors)

In [53]:
add_time_lagged(['fews_ipc'], end=21, diff=3)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

test_splits = [
    ((2010,7), (2011, 7)), 
    ((2011,7), (2012, 7)),
    ((2012,7), (2013, 7)), 
    ((2013,7), (2014, 7)), 
    ((2014,7), (2015, 7)), 
    ((2015,7), (2016, 7)), 
    ((2016,7), (2017, 7)), 
    ((2017,7), (2018, 7)),
    ((2018,7), (2019, 7)), 
    ((2019,2), (2020, 2)),
]
train_splits = [
    ((2009,7), (2010,4)),
    ((2009,7), (2011,1)),
    ((2009,7), (2011,10)),
    ((2009,7), (2012,7)),
    ((2009,7), (2013,7)),
    ((2009,7), (2014,1)),
    ((2009,7), (2015,1)),
    ((2009,7), (2015,10)),
    ((2009,7), (2016,10)),
    ((2009,7), (2017,2))]
dev_splits = [
    ((2010,4), (2010, 7)),
    ((2011,1), (2011, 7)),
    ((2011,10), (2012, 7)),
    ((2012,7), (2013, 7)),
    ((2013,4), (2014, 7)),
    ((2014,1), (2015, 7)),
    ((2015,1), (2016, 7)),
    ((2015,10), (2017, 7)),
    ((2016,10), (2018, 7)),
    ((2017,2), (2019, 2)),
]
regr = RandomForestRegressor(max_features='auto', n_estimators=100, 
                             min_samples_split=0.5, min_impurity_decrease=0.001, random_state=0)

features = {
    'traditional': time_series[
        ['{}_{}'.format('fews_ipc', t) for t in range(3,21,3)] + 
        ['{}_{}'.format(f, t) for f, t in zip(t_variant_traditional_factors, range(3,9))] + 
        t_invariant_traditional_factors
    ], 
    'news': time_series[
        ['{}_{}'.format('fews_ipc', t) for t in range(3,21,3)] +
        ['{}_{}'.format(f, t) for f, t in zip(news_factors, range(3,9))]
    ], 
    'traditional+news': time_series[
        ['{}_{}'.format('fews_ipc', t) for t in range(3,21,3)] +
        ['{}_{}'.format(f, t) for f, t in zip(t_variant_traditional_factors, range(3,9))] + 
        t_invariant_traditional_factors +
        ['{}_{}'.format(f, t) for f, t in zip(news_factors, range(3,9))]
    ], 
}

labels_df = time_series['fews_ipc']

def get_time_split(df, start, end):
    return df[df['year'] >= start[0] & df['month'] >= start[1] & df['year'] <= end[0] & df['month'] <= end[1]]

for train, dev, test in zip(train_splits, dev_splits, test_splits):
    for f, D in features.items():
        X = get_time_split(D, train[0], train[1])
        y = get_time_split(labels_df, test[0], test[1])
        regr.fit(X, y)
        X_test = get_time_split(D, test[0], test[1])
        preds = regr.predict(X_test)
        labels = get_time_split(labels_df, test[0], test[1])
        rmse = mean_squared_error(labels, preds, squared=True)
        print (test, f, rmse)

[-2.50699856]
